In [17]:
import pymysql
import pandas as pd
import lxml

In [18]:
## database password
## hidden
pw = '04250629'

In [19]:
conn = pymysql.connect(
    host='localhost',
    user='root',
    password=pw,
    db='stock',
    charset='utf8'
    )

# create cursor
## create company_info and daily_price table if not exists
with conn.cursor() as curs:
    sql = """
    CREATE TABLE IF NOT EXISTS company_info (
        code VARCHAR(20),
        company VARCHAR(100),
        last_update DATE,
        PRIMARY KEY (code)
    )
    """

    curs.execute(sql)

    sql = """
    CREATE TABLE IF NOT EXISTS daily_orice(
        code VARCHAR(20),
        date DATE,
        open BIGINT,
        high BIGINT,
        low BIGINT,
        close BIGINT,
        diff BIGINT,
        volume BIGINT,
        PRIMARY KEY (code, date)
    )
    """
    curs.execute(sql)
    curs.close()
        
    # commit
conn.commit()

In [20]:
url="https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13"
krx = pd.read_html(url, header=0)[0]
krx = krx[['종목코드', '회사명']]
krx = krx.rename(columns={'종목코드':'code', '회사명':'company'})
krx.code = krx.code.map('{:06d}'.format)
krx.head()                               

,code,company
0,095570,AJ네트웍스
1,138930,BNK금융지주
2,155660,DSR
3,078930,GS
4,294870,HDC현대산업개발


In [27]:
from datetime import datetime
codes = dict()
sql = "SELECT * FROM company_info"
df = pd.read_sql(sql, conn)
for idx in range(len(df)):
    ## update codes dictionary using code and company_name
    codes[df['code'].values[idx]] = df['company'].values[idx]
with conn.cursor() as curs:
    sql = """
    SELECT max(last_update) FROM company_info
    """
    curs.execute(sql)
    ## get recent update date
    rs = curs.fetchone()
    today = datetime.today().strftime('%Y-%m-%d')

    if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
        for idx in range(len(krx)):
            code = krx.code.values[idx]
            company = krx.company.values[idx]
            sql = f"REPLACE INTO company_info (code, company, last_update) VALUES ('{code}', '{company}', '{today}')"
            curs.execute(sql)
            codes[code] = company
            tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
            print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info VALUES ({code}, {company}, {today})")
        conn.commit()
        print('')
    curs.close()



/var/folders/dt/q9f0jq252ld5kmwg8rqf4pgw0000gn/T/ipykernel_91990/2405500686.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


In [41]:
from bs4 import BeautifulSoup
import urllib
import requests

# setting UA header
header = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}

# request url
url = 'http://finance.naver.com/item/sise_day.nhn?code=005930'
request = urllib.request.Request(url, headers=header)

# get html
doc =urllib.request.urlopen(request)
html = BeautifulSoup(doc, "lxml")

# parsing
## find the last page
pgrr = html.find('td', class_='pgRR')
s = str(pgrr.a['href']).split('=')
lastpage = s[-1]

# make dataframe
df = pd.DataFrame()

# crawling
## crawling min(lastpage, 10) pages
pages = min(int(lastpage), 10)
for page in range(1, pages+1):
    page_url = '{}&page={}'.format(url, page)
    # append daily prices into dataframe
    df = df.append(pd.read_html(requests.get(page_url, headers=header).text)[0])
    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
    print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.format(tmnow, codes['005930'], '005930', page, pages), end="\r")

df = df.rename(columns={'날짜':'date', '종가':'close', '전일비':'diff', '시가':'open', '고가':'high', '저가':'low', '거래량':'volume'})
df = df.dropna()
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)
df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]

df.head()



/var/folders/dt/q9f0jq252ld5kmwg8rqf4pgw0000gn/T/ipykernel_91990/2023275707.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_html(requests.get(page_url, headers=header).text)[0])
/var/folders/dt/q9f0jq252ld5kmwg8rqf4pgw0000gn/T/ipykernel_91990/2023275707.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_html(requests.get(page_url, headers=header).text)[0])
/var/folders/dt/q9f0jq252ld5kmwg8rqf4pgw0000gn/T/ipykernel_91990/2023275707.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_html(requests.get(page_url, headers=header).text)[0])
/var/folders/dt/q9f0jq252ld5kmwg8rqf4pgw0000gn/T/ipykernel_91990/2023275707.py:31: FutureWarning: The frame.append 

/var/folders/dt/q9f0jq252ld5kmwg8rqf4pgw0000gn/T/ipykernel_91990/2023275707.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_html(requests.get(page_url, headers=header).text)[0])
/var/folders/dt/q9f0jq252ld5kmwg8rqf4pgw0000gn/T/ipykernel_91990/2023275707.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_html(requests.get(page_url, headers=header).text)[0])
/var/folders/dt/q9f0jq252ld5kmwg8rqf4pgw0000gn/T/ipykernel_91990/2023275707.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_html(requests.get(page_url, headers=header).text)[0])
/var/folders/dt/q9f0jq252ld5kmwg8rqf4pgw0000gn/T/ipykernel_91990/2023275707.py:31: FutureWarning: The frame.append 

/var/folders/dt/q9f0jq252ld5kmwg8rqf4pgw0000gn/T/ipykernel_91990/2023275707.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_html(requests.get(page_url, headers=header).text)[0])
/var/folders/dt/q9f0jq252ld5kmwg8rqf4pgw0000gn/T/ipykernel_91990/2023275707.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_html(requests.get(page_url, headers=header).text)[0])


,date,open,high,low,close,diff,volume
1,2023.06.02,71700,72200,71600,72200,1300,11716021
2,2023.06.01,70900,71600,70600,70900,500,14669296
3,2023.05.31,72400,72500,71000,71400,900,25666087
4,2023.05.30,71300,72300,71200,72300,2000,27476897
5,2023.05.26,69800,70400,69500,70300,1500,19549511


In [22]:
# curs.close()
# conn.close()